In [1]:
%run parseData.ipynb
#%run dataClass.ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd

In [32]:
class GestureModel(nn.Sequential):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.actv1 = nn.ReLU()

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=4, kernel_size=1)
        self.actv2 = nn.ReLU()

        self.do1 = nn.Dropout(0.2)

        self.pool1 = nn.MaxPool1d(2)

        self.flat = nn.Flatten()

        self.linear1 = nn.Linear(in_features=35, out_features=64)
        self.actv3 = nn.ReLU()

        self.linear2 = nn.Linear(in_features=64, out_features=4)
        self.actv4 = nn.Softmax()
    
    def forward(self, input):
        input = self.actv1(self.conv1(input))
        input = self.actv2(self.conv2(input))
        input = self.do1(input)
        input = self.pool1(input)
        input = self.flat(input)
        input = self.actv3(self.linear1(input))
        input = self.actv4(self.linear2(input))
        return input

In [3]:
path = "./data/"

In [108]:
def MultiplyVector(vector_data):
    multiplied_list = []
    for data in vector_data:
        multiplied_vectors = []
        for vector in data:
            multiplied = np.prod(vector)
            multiplied_vectors.append(multiplied)
        multiplied_list.append(multiplied_vectors)
    return multiplied_list

In [4]:
all_data = read_data(path)
data = torch.from_numpy(all_data)

#data, data.shape

In [5]:
labels = data[:, :1].long()
#labels_encoding = labels.unsqueeze(1)

#labels_encoding = F.one_hot(labels)
#labels.reshape((-1,))

labels_encoded = torch.zeros(labels.shape[0], 4)
labels_encoded.scatter_(1, labels, 1.0)

labels_encoded[0].dtype, labels_encoded.shape

#df = pd.DataFrame(labels_encoding)

#labels, labels.shape

(torch.float32, torch.Size([141, 4]))

In [6]:
rotations = data[:, 2:].double()

#rotations, rotations.shape

In [7]:
split = round(len(data) * 0.8)
split

113

In [8]:
train_data = rotations[:split]
train_labels = labels_encoded[:split]

#train_data, train_data.shape, train_labels, train_labels.shape

In [9]:
test_data = rotations[split:]
test_labels = labels_encoded[split:]

#test_data, test_data.shape, test_labels, test_labels.shape

In [15]:
train_dataset = TensorDataset(train_data.float(), train_labels)
test_dataset = TensorDataset(test_data.float(), test_labels)

train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

In [45]:
train_loader.dataset.tensors[0].shape[1:3]

torch.Size([72])

In [41]:
model = GestureModel()
model.float()
optimizer = optim.SGD(model.parameters(), lr=0.0001)
#optimizer = optim.

n_epochs = 100

for epoch in range(n_epochs):

    total_loss = 0

    for inputs, labels in train_loader:
        pred = model(inputs)

        loss = (pred - labels) ** 2
        print(loss)

        loss.sum().backward()

        total_loss += loss

        optimizer.step()
        optimizer.zero_grad()
    acc = 0
    count = 0
    for inputs, labels in test_loader:
        pred = model(inputs)
        #print(inputs)
        #print(labels)
        #print(pred.shape)
        acc += (torch.argmax(pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count

    optimizer.step()
    optimizer.zero_grad()
    #print("Epoch " + str(epoch) + " model accuracy " + str(acc * 100) + " loss " + str(total_loss))

C:\Users\VR-LAB2\AppData\Local\Temp\ipykernel_2288\3273693136.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = self.actv4(self.linear2(input))


tensor([[0.5297, 0.0593, 0.0532, 0.0643],
        [0.5309, 0.0606, 0.0527, 0.0639],
        [0.5314, 0.0596, 0.0538, 0.0639],
        [0.5313, 0.0606, 0.0533, 0.0634]], grad_fn=<PowBackward0>)
tensor([[0.0720, 0.0610, 0.5933, 0.0650],
        [0.0738, 0.0601, 0.5937, 0.0644],
        [0.0737, 0.0594, 0.5916, 0.0645],
        [0.0735, 0.0606, 0.5914, 0.0634]], grad_fn=<PowBackward0>)
tensor([[0.0728, 0.0604, 0.5948, 0.0654],
        [0.0740, 0.0601, 0.5921, 0.0637],
        [0.0722, 0.0606, 0.5939, 0.0654],
        [0.0735, 0.0605, 0.5914, 0.0634]], grad_fn=<PowBackward0>)
tensor([[0.0722, 0.0609, 0.5950, 0.0654],
        [0.0741, 0.0602, 0.5930, 0.0637],
        [0.0724, 0.0604, 0.5940, 0.0654],
        [0.0735, 0.0605, 0.5913, 0.0634]], grad_fn=<PowBackward0>)
tensor([[0.5299, 0.0593, 0.0529, 0.0647],
        [0.5305, 0.0604, 0.0526, 0.0642],
        [0.5303, 0.0596, 0.0540, 0.0634],
        [0.5313, 0.0605, 0.0534, 0.0634]], grad_fn=<PowBackward0>)
tensor([[0.5300, 0.0593, 0.0533, 0.